In [ ]:
import os,sys
ROOT_DIR = os.path.abspath("../../")
DATA_DIR = os.path.join(ROOT_DIR, 'data/processed')
sys.path.append(ROOT_DIR)

from src.data import helper
from src.data.cityscapeDataset import CityscapeDataset
from src.data.cityscapeConfig import CityscapeConfig


import numpy as np
import pandas as pd
import tensorflow as tf
import random
import matplotlib.pyplot as plt

# model import
from models.maskrcnn.mrcnn import visualize
from models.maskrcnn.mrcnn import model_new as modellib
from models.maskrcnn.mrcnn.model import log
from models.maskrcnn.mrcnn import config
from models.maskrcnn.mrcnn.config import Config
from models.maskrcnn.mrcnn import utils

In [ ]:
# create new training configurations
cconfig = CityscapeConfig()

dataset = CityscapeDataset()
dataset.load_dataset(DATA_DIR,'train')
dataset.prepare()

val_set = CityscapeDataset()
val_set.load_dataset(DATA_DIR,'val')
val_set.prepare()

In [ ]:
id = random.choice(dataset.image_ids)
print(id)
img = dataset.load_image(id)
masks,labels,obs = dataset.load_mask(id)
names = dataset.class_from_source_map

visualize.display_top_masks(img,
                            masks,
                            labels,
                            names,
                            limit=4)


visualize.display_top_masks(img,
                            masks,
                            obs,
                            {0:'BG',1:'obstruction'},
                            limit=2)

In [ ]:
COCO_MODEL_PATH = os.path.join(ROOT_DIR,'models/maskrcnn/mask_rcnn_coco.h5')
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
tf.keras.backend.clear_session()
# define a directory to store the tf logs and weights
temp_log = os.path.join(ROOT_DIR,'data/external')
# define existing weights
model_path = os.path.join(ROOT_DIR,'models/maskrcnn/mask_rcnn_coco.h5')


# construct the model with custom configuration
model = modellib.MaskRCNN(mode='training',
                            config=cconfig, 
                            model_dir=temp_log)

model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
# train the model
model.train(dataset, val_set, 
            learning_rate=cconfig.LEARNING_RATE, 
            epochs=2, 
            layers='heads',
        )

In [ ]:
model.keras_model.summary()

# Inference

In [ ]:
# model_path = os.path.join(ROOT_DIR,'models/maskrcnn/mask_rcnn_sidewalks_0001.h5')
try:
  model_path = model.find_last()
except:
  # model_path = '//Users/csea/Desktop/deep-learning-final-project-project-sidewalk/models/maskrcnn/logs/3_BCE/mask_rcnn_sidewalks_0010.h5'
  model_path ='/Users/csea/Desktop/logs_nicholas/mask_rcnn_sidewalks_0020_fixed_dense.h5'
  model_path ='/Users/csea/Desktop/mask_rcnn_sidewalks_0004_shared.h5'
  # model_path ='/Users/csea/Downloads/mask_rcnn_sidewalks_0002.h5'
  # model_paty = '/Users/csea/Desktop/logs_nicholas/new///mask_rcnn_sidewalks_0020_3Dense_BCE.h5'
print(f'Found model at {model_path}')
class InferenceConfig(CityscapeConfig):
  GPU_COUNT =1 
  IMAGES_PER_GPU = 1
  DETECTION_MIN_CONFIDENCE = .5
  IMAGE_MAX_DIM = 512
  IMAGE_MIN_DIM = 512
  USE_MINI_MASK = True

inference_config = InferenceConfig()
temp_log = os.path.join(ROOT_DIR,'data/external')

tf.keras.backend.clear_session()
model = modellib.MaskRCNN(mode = 'inference',
                          config = inference_config,
                          model_dir=temp_log)
# model.load_weights(model_path, by_name=True, )
tf.keras.Model.load_weights(model.keras_model, model_path, by_name=True)


In [ ]:
dataset = val_set
ids = random.choice(dataset.image_ids)
# ids = 272
print(ids)
# ids = 297
image = dataset.load_image(ids)
results = model.detect([image],verbose=1)
# display preferences
def get_ax(rows=1, cols=1, size=10):
  _,ax = plt.subplots(rows,cols,figsize=(size*cols,size*rows))
  return ax

# captions = helper.generate_captions_obstructed(r['class_ids'], r['obstructions'],labels_dict = val_set.rev_mapping)
r = results[0]

visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                           list(dataset.class_from_source_map.values()), r['scores'], 
                           ax=get_ax(), title='MaskRCNN Segmentations', show_bbox=False)


idx = np.where(r['obstructions_ids']==1)[0]

b,m,l = helper.filter_obstructions(idx, r['rois'], r['masks'], r['class_ids'])
visualize.display_instances(image, b,m,l, 
                           list(dataset.class_from_source_map.values()), r['obstructions_scores'][idx], 
                           ax=get_ax(),title='MaskRCNN Obstructions ', show_bbox=False)
                           

In [ ]:
# image = dataset.load_image(ids)
# ids = 122
gt_mask, gt_class_ids, gt_obs = dataset.load_mask(ids)
gt_bbox = utils.extract_bboxes(gt_mask)

def get_ax(rows=1, cols=1, size=10):
  _,ax = plt.subplots(rows,cols,figsize=(size*cols,size*rows))
  return ax

_ = np.where(gt_obs==1)[0]
visualize.display_instances(image, gt_bbox,gt_mask, gt_class_ids, 
                            list(dataset.class_from_source_map.values()), ax=get_ax(), 
                            show_bbox=False, title='GT Segmentations')

gt_bbox_filtered,gt_mask_filtered,gt_class_ids_filtered = helper.filter_obstructions(_, gt_bbox,gt_mask,gt_class_ids)
visualize.display_instances(image, gt_bbox_filtered,gt_mask_filtered, gt_class_ids_filtered,
                            list(dataset.class_from_source_map.values()), 
                            ax=get_ax(), show_bbox=False, title = 'GT obstructions')

In [ ]:
visualize.display_differences(image, gt_bbox,gt_class_ids, gt_mask,
                                r['rois'],r['class_ids'],r['scores'],r['masks'],list(dataset.class_from_source_map.values()))

# Evaluation

## Testing AP functions

In [ ]:
mAP,precisions, recalls, overlaps = utils.compute_ap(gt_bbox,gt_class_ids, gt_mask,
                                r['rois'],r['class_ids'],r['scores'],r['masks'])

visualize.plot_precision_recall(mAP, precisions, recalls)